In [141]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import tensorflow as tf
import keras as keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM

In [142]:
# Dicts to identify which countries go with which tickers (all are 10-year Govt yields)
codes = {}
codes['US'] = 'USGG10YR'
codes['Germany'] = 'GDBR10'
codes['UK'] = 'GUKG10'
codes['France'] = 'GFRN10'
codes['Australia'] = 'GACGB10'
codes['Canada'] = 'GCAN10YR'
codes['New Zealand'] = 'GNZGB10'
codes['Japan'] = 'JGBS10'
codes['Switzerland'] = 'GSWISS10'
codes['Norway'] = 'GNOR10YR'
codes['Italy'] = 'GBTPGR10'

codes_back = {}
for key, value in codes.items():
    codes_back[value] = key

sheet_names = pd.ExcelFile('G10_RV.xlsx').sheet_names

# Combining data into single df
for i, x in enumerate(sheet_names[:11]):
    if i == 0:
        df = pd.read_excel('G10_RV.xlsx', sheet_name=x)[['Date', 'Last Price']]
        df.columns = ['Date', x]
    else:
        new_df = pd.read_excel('G10_RV.xlsx', sheet_name=x)[['Date', 'Last Price']]
        new_df.columns = ['Date', x]
        df = df.merge(new_df, on='Date', how='outer')

# Filling in missing days with previous observations, defining which columns are rates we want
df = df.set_index('Date')
df = df.resample('D').asfreq()
df = df.ffill()
df = df[::-1].dropna()
rates_tickers = df.columns[:11]

df

,USGG10YR,GDBR10,GUKG10,GFRN10,GACGB10,GCAN10YR,GNZGB10,JGBS10,GSWISS10,GNOR10YR,GBTPGR10
Date,,,,,,,,,,,
2024-04-12,4.5216,2.359,4.137,2.865,4.266,3.649,4.839,0.864,0.739,3.707,3.762
2024-04-11,4.5865,2.463,4.201,2.962,4.256,3.730,4.768,0.854,0.782,3.799,3.873
2024-04-10,4.5435,2.435,4.148,2.926,4.118,3.695,4.638,0.801,0.738,3.740,3.807
2024-04-09,4.3616,2.371,4.029,2.861,4.173,3.556,4.718,0.786,0.735,3.719,3.751
2024-04-08,4.4198,2.435,4.085,2.924,4.197,3.625,4.692,0.792,0.772,3.767,3.829
...,...,...,...,...,...,...,...,...,...,...,...
2010-01-09,3.8297,3.385,4.063,3.572,5.681,3.600,5.780,1.359,2.005,4.079,4.078
2010-01-08,3.8297,3.385,4.063,3.572,5.681,3.600,5.780,1.359,2.005,4.079,4.078
2010-01-07,3.8235,3.370,4.052,3.581,5.646,3.637,5.792,1.345,2.003,4.122,4.094


In [143]:
t_values = [1,5,10,25,50,100]

def nn_multi(target, t):
    target_t = f'{target}_{t}'
    data = df[rates_tickers].copy()
    for ticker in rates_tickers:
        for x in t_values:
            data[f'{ticker}_{x}'] = data[ticker].diff(-x)
    data = data.dropna()
    data_training = data[data.index < '2023-1-1'].copy()
    data_testing = data[data.index >= '2023-1-1'].copy()
    training_X = data_training[[x for x in data_training if '_' in x and x != target_t]]
    training_y = data_training[target_t]
    testing_X = data_testing[[x for x in data_testing if '_' in x and x != target_t]]
    testing_y = data_testing[target_t]
    scaler = StandardScaler()
    training_X = scaler.fit_transform(training_X)
    testing_X = scaler.transform(testing_X)

    model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=[training_X.shape[1]]),
            Dropout(0.3),
            tf.keras.layers.Dense(32, activation='relu'),
            Dropout(0.3),
            tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    model.fit(training_X, training_y, epochs=200, validation_split=0.2, callbacks=[early_stopping])
    testing_prediction = model.predict(testing_X)
    r2 = round(r2_score(testing_y, testing_prediction), 2)
    data_testing['c_prediction'] = testing_prediction
    prediction = data_testing[[target, target_t, 'c_prediction']].copy()
    prediction['prediction'] = prediction[target].shift(-t) + prediction['c_prediction']
    prediction = prediction[[target, 'prediction']].dropna()
    return prediction, r2

In [144]:
def t_eval(t):
    r2s = pd.DataFrame()
    r2s['Target'] = list(codes.keys())
    r2s['r2: Multi Neural Network'] = r2s['Target'].apply(lambda x: nn_multi(codes[x], t)[1])
    r2s = r2s.set_index('Target')
    print(r2s['r2: Multi'].mean())
    return r2s

t_eval(10)

Epoch 1/200
117/117 [==============================] - 0s 1ms/step - loss: 0.2906 - val_loss: 0.0382
Epoch 2/200
117/117 [==============================] - 0s 710us/step - loss: 0.0629 - val_loss: 0.0236
Epoch 3/200
117/117 [==============================] - 0s 712us/step - loss: 0.0340 - val_loss: 0.0174
Epoch 4/200
117/117 [==============================] - 0s 694us/step - loss: 0.0223 - val_loss: 0.0144
Epoch 5/200
117/117 [==============================] - 0s 695us/step - loss: 0.0166 - val_loss: 0.0138
Epoch 6/200
117/117 [==============================] - 0s 696us/step - loss: 0.0141 - val_loss: 0.0121
Epoch 7/200
117/117 [==============================] - 0s 717us/step - loss: 0.0115 - val_loss: 0.0114
Epoch 8/200
117/117 [==============================] - 0s 763us/step - loss: 0.0097 - val_loss: 0.0109
Epoch 9/200
117/117 [==============================] - 0s 762us/step - loss: 0.0094 - val_loss: 0.0103
Epoch 10/200
117/117 [==============================] - 0s 755us/step - los

,r2: Multi
Target,
US,0.91
Germany,0.92
UK,0.84
France,0.95
Australia,0.86
Canada,0.87
New Zealand,0.84
Japan,0.63
Switzerland,0.79
